# 2022-12-06

## Performance Summary
Inital p1 & p2: 2.67 ms ± 28.4 µs

## Initial solution
Woke up att 6:00am (~Midnight UTC-5) and did it on time. 
```
      --------Part 1--------   --------Part 2--------
Day       Time   Rank  Score       Time   Rank  Score
  7   05:33:29  24961      0   05:51:29  24067      0
```

### Both puzzles

This took way longer than expected, embarrassingly 6h. However, todays solution is very class like and that I like about it. We iterate through each input row and build a tree that represents the file structure. During runtime of building the tree we also calculate directory sizes without regard to child directories. We also store the type of object in the tree using metadata `type = _dir | _file`. We use this later when we recursivly traverse our tree to calculate the sums of directories. For P2, during the traversal, we store all cumulated-directory-sizes in a list and find the minimum that is larger than our requirement.

In [105]:
from functools import reduce
from operator import getitem


rows = [x.split(" ") for x in open("data/day-7.txt").read().splitlines()]

In [106]:
class FileSystem():
    def __init__(self):
        self.tree = dict()
        self.links = dict()
        self.current_dir = self.tree
        self.path = []
        self.p1sum = 0
        self.dirsizes = []
    def cd(self, dir):
        if dir == "..":
            self.path.pop()
            self.current_dir = reduce(getitem, self.path, self.tree)
        else:
            self.path.append(dir)
            self.current_dir[dir] = dict(_type="dir")
            self.current_dir = self.current_dir[dir]
    def create(self, file, **kwargs):
        self.current_dir[file] = dict(kwargs)
    def read_input(self, *args):
        match args[0]:
            case "$":
                match args[1]:
                    case "cd": self.cd(args[2])
                    case "ls": self.current_dir["_itemsizes"] = 0
            case "dir":
                self.create(args[1], _type="dir")
            case _:
                self.create(args[1],  _type="file", _size=args[0])
                self.current_dir["_itemsizes"] += int(args[0])
    def build(self):
        for row in rows: self.read_input(*row) 
    def traverse(self, tree):
        subdirs = [self.traverse(tree[x]) for x in tree if type(tree[x]) == dict and tree[x]["_type"] == "dir"]
        dirsize = tree["_itemsizes"] + sum(subdirs)
        self.p1sum += dirsize if dirsize <= 100000 else 0
        self.dirsizes.append(dirsize)
        tree["_dirsize"] = dirsize
        return dirsize
    def traverse_tree(self):
        self.outersum = self.traverse(self.tree["/"])
        self.p2req = 30000000 - (70000000 - self.outersum)
        self.p2ans = min([x for x in self.dirsizes if x >= self.p2req])
        return self.outersum
    
system = FileSystem(); system.build(); system.traverse_tree(); system.traverse_tree()

print(f"Outermost sum: {system.outersum}")
print(f"P1 sum: {system.p1sum}")
print(f"We need to remove: {system.p2req}, smallest directory: {system.p2ans}")

Outermost sum: 46728267
P1 sum: 2781648
We need to remove: 6728267, smallest directory: 7490863


In [107]:
assert(system.traverse_tree() == sum([int(x[0]) for x in rows if x[0].isdigit()]))
assert(1084924+164331+102120 == system.tree["/"]["cvt"]["chhdjtlw"]["_dirsize"])

# Learnings

So today took a very long time. I spent a lot of time on figuring out just how to build a tree that one could navigate up and down in. Once that was in place, I ran into issues with how I calculated the non-cumulative-directory size. These issues took quite a while to solve. The recursion part went relatively quick.

- I need to learn how to work with trees in python.

# Runtimes

In [104]:
%%timeit
system = FileSystem(); system.build(); system.traverse_tree(); system.traverse_tree()

2.67 ms ± 28.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
